# Introduction to OSPF
<i>Adapted for use with FABRIC from [OSPF](https://www.cs.unc.edu/Research/geni/geniEdu/06-Ospf.html)</i>
    
In this tutorial you will experiment with the OSPF routing protocol.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/OSPF/figures/OSPFTop.png?raw=true"><br>
OSPF stands for Open Shortest Path First, and is currently the most widely used interior gateway protocol in the world. OSPF is a dynamic routing protocol, meaning changes are accounted for automatically. Hello messages are used to discover neighbors and find breaks. By default, hello messages are sent by OSPF routers every 10 seconds on point to point segments.
<br><br>Each OSPF router maintains a Link State Database (LSDB) containing Link State Advertisements (LSAs). Newly discovered neighbors exchange their LSDBs with Database Description (DDs) messages.
<br><br>OSPF uses Router Dead Intervals, the maximum time a router may be "silent" before being declared "down". OSPF uses Djikstra's Shortest path algorithm to compute routes.
<br><br>This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = "OSPF"
slice = fablib.get_slice(slice_name)
slice.list_nodes()


## 2. Guided Experiment

<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/OSPF/figures/ospf.png?raw=true"><br>

### 2.1 Router Interface
1. Login to each node
2. Open the Quagga shell
Quagga provides an integrated user interface shell called vtysh, a cisco-like CLI, connecting to the underlying daemons (zebra and ospfd). <br>Open the shell on each node by typing:
<br>```sudo vtysh```
3. Look at the OSPF neighbors for a node by typing the following into a vtysh console:
<br>```show ip ospf neighbor```
<br>The output shows the status of neighbors including details of the connections, such as the Dead time, wichh indicates a countdown to refresh the connection.

4. The routing table for a node can be seen by typing the following into a vtysh console:
<br>```show ip ospf route```
<br>This table shows how packets would be forwarded from the source to a given available destination.
5. Exiting the console can be done by executing:
<br>```exit```


### 2.2 Dead Link
1. Login to a node
2. Show available interfaces with:
<br>```ip addr```
3. Take down one of the links (enp7s0 or enp87s0), keep in mind the IP address associated:
<br>```sudo ip link set dev <interface> down```

4. Quickly switch back to vtysh shell and reissue:
<br>```show ip ospf neighbor```
5. Reissue the command multiple times and watch the Dead Time count down until the neighbor is removed from the table. The other link will also count down, but resets when receiving a hello message.
6. Exit the console and bring the interface back up by typing:
<br>```sudo ip link set dev <interface> up```
7. List the OSPF neighbors once more to see the link listed.

### 2.3 Route Change
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/OSPF/figures/deadLink.png?raw=true"><br>
1. Login to node 'A'
2. Run the following command:
<br>```traceroute 10.1.3.1```
<br>Traceroute will determine the route that packets are taking through the network. Running the command on node A will show packets traveling through 10.1.4.1 (node D) to 10.1.3.1 (node C).
3. Take down the interface to node D (enp8s0):
<br>```sudo ip link set dev <interface> down```

4. Run traceroute again:
<br>```traceroute 10.1.3.1```
<br>The output should now show traffic going through 10.1.1.2 (node B).
5. Bring the route to node D (enp8s0) back up:
<br>```sudo ip link set dev <interface> up```
6. Repeatedly run traceroute and eventually the route through node D will reappear.


## 3.  Assignment
### Part 1
From A ping one of C's interfaces. See if you can trick OSPF into failing to route packets momentarily (causing ping to hang) even though there is a path from A to C.
1. Briefly describe how you did it?

### Part 2
The hello interval and dead interval values can be configured in the ospfd configuration located at /etc/quagga/ospfd.conf. Modify this file on each of the nodes, adding two lines below each interface like this:
<br> `interface eth1`
<br> `ip ospf hello-interval 2`
<br> `ip ospf dead-interval 5`

This changes the hello interval to 2 seconds and the dead  interval to 5 seconds. change these values on each of the 4 nodes, and then restart the quagga by typing the following command: ```sudo /etc/init.d/quagga restart```
finally enter the router interface by typing: ```vtysh```

then redo 2.2 and 2.3 to answer the following questions:

1. From your experience in the main part of the experiment, before you made changes, what is the default value of the dead-interval for this implementation of OSPF?
2. After making the above changes, what is different in how OSPF responds to broken links?
3. What types of networks would shorter hello-interval and dead interval values be more suited for?
4. What is one downside of using shorter hello-intervals and dead-intervals?


## Cleanup Resources
Once you have completed the assignment shut down the slice

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")